In [ ]:
import os

In [ ]:
image_root_dir = "./data"
output_dirname = "hand"

input_dirs = [os.path.join(image_root_dir, dirname) for dirname in os.listdir(image_root_dir) if dirname != output_dirname]
class_names = os.listdir(input_dirs[0])
output_dirs = {class_name : os.path.join(image_root_dir, output_dirname, class_name) for class_name in class_names}

source_images = {class_name : [] for class_name in class_names}
for image_dir in input_dirs:
    for class_name in class_names:
        path = os.path.join(image_dir, class_name)
        source_images[class_name] += [os.path.join(path, image) for image in os.listdir(path)]
     
    
[os.makedirs(output_dir) for output_dir in output_dirs.values()]

In [ ]:
import random
import string
import cv2
import numpy as np

def gen_random_name(n):
    randlst = [random.choice(string.ascii_letters + string.digits) for i in range(n)]
    return ''.join(randlst)

def gen_square_image(image):
    tmp = image.copy()
    height, width = image.shape[:2]
    size, limit = max(height, width), min(height, width)
    start = (size - limit) // 2
    end   = (size + limit) // 2
    new_image = cv2.resize(np.full((1, 1, 3), 255, dtype=np.uint8), (size, size))
    if height > width:
        new_image[:, start:end] = tmp
    else:
        new_image[start:end, :] = tmp
    return new_image

def save_image(image, class_name):
    output_name = os.path.join(output_dirs[class_name], gen_random_name(8) + ".png")
    cv2.imwrite(output_name, image)

In [ ]:
def translate_and_save_image(image_path, class_name):
    image = cv2.imread(image_path)
    image = gen_square_image(image)
#     save_image(image, class_name)
    center = (image.shape[0]//2, image.shape[1]//2)
#     for deg in range(0, 360, 30):
    for deg in [30]:
        trans = cv2.getRotationMatrix2D(center, deg, 1.0)
        rotated_image = cv2.warpAffine(image.copy(), trans, dsize=(640, 640), borderValue=(255, 255, 255))
        save_image(rotated_image, class_name)
#     for direction in [cv2.ROTATE_90_CLOCKWISE, cv2.ROTATE_90_COUNTERCLOCKWISE, cv2.ROTATE_180]:
    for direction in [cv2.ROTATE_90_CLOCKWISE]:
        rotated_image = cv2.rotate(image.copy(), direction)
        save_image(rotated_image, class_name)


In [ ]:
for class_name in class_names:
    images = source_images[class_name]
    for image in images:
        translate_and_save_image(image, class_name)